In [2]:
import pandas as pd
import numpy as np
import os
from random import sample

from PIL import Image
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import mean_squared_error

from visual import tools, evaluation

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
path_to_model =  '/home/lichi/jupiter/imaginarium/pretrained_models/coco.npz'
path_to_model_f30k =  '/home/lichi/jupiter/imaginarium/pretrained_models/f30k.npz'
model = tools.load_model(path_to_model = path_to_model)
model_f30k = tools.load_model(path_to_model = path_to_model_f30k)
#evaluation.evalrank(model, data='f8k', split='test')

/home/lichi/jupiter/imaginarium/pretrained_models/coco.npz
Loading dictionary...
Creating inverted dictionary...
Loading model options...
Loading model parameters...
Compiling sentence encoder...
Compiling image encoder...
Packing up...
/home/lichi/jupiter/imaginarium/pretrained_models/f30k.npz
Loading dictionary...
Creating inverted dictionary...
Loading model options...
Loading model parameters...
Compiling sentence encoder...
Compiling image encoder...
Packing up...


#### Читаем картинки, строим эмбеддинги и извлекаем из них VGG19 фичи

In [4]:
base_model = VGG19(weights='imagenet', include_top=True)
model_vgg19 = Model(input=base_model.input, output=base_model.get_layer('fc2').output)

paths_images = os.listdir('images')
img_list = []
feat_list = []
for path in paths_images:
    print path
    if path!='.ipynb_checkpoints':
        img=mpimg.imread('images/' + path,0)
        img_crop = img[30:-30,30:-211]
        h = img_crop.shape[0]
        w = img_crop.shape[1]
        img16 = []
        feat16 = []
        for i in np.arange(0,1,0.5):
            for j in np.arange(0,1,0.25):
                pic = img_crop[int(h*i):int(h*(i+1/2.)),int(w*j):int(w*(j+1/4.))]
                pil = Image.fromarray(pic)
                path_save = 'images_preprocessed/'+path[:-5]+str(i)+str(j)+ ".jpeg"
                pil.save(path_save)
                img = image.load_img(path_save, target_size=(224, 224))
                pic = image.img_to_array(img)
                pic = np.expand_dims(pic, axis=0)
                pic = preprocess_input(pic)
                features = model_vgg19.predict(pic)
                img16.append(img)
                feat16.append(features)
                
        img_list += img16
        feat_list += feat16

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("fc..., inputs=Tensor("in...)`
  


img 17.jpeg
img 4.jpeg
img 16.jpeg
img 3.jpeg
img 8.jpeg
img 20.jpeg
img 23.jpeg
img 18.jpeg
img 15.jpeg
img 13.jpeg
img 2.jpeg
img 22.jpeg
.ipynb_checkpoints
img 14.jpeg
img 11.jpeg
img 9.jpeg
img 7.jpeg
img 21.jpeg
img 10.jpeg
img 6.jpeg
img 19.jpeg
img 12.jpeg
img 5.jpeg


In [7]:
feat_list = [x.reshape(4096) for x in feat_list]
image_vectors = tools.encode_images(model, feat_list)
image_vectors_f30k = tools.encode_images(model_f30k, feat_list)

#### Берем топ-частотных слов

In [8]:
#words = pd.read_csv('words/4k_word_list.txt', sep = ',', header = -1)[0]
words = pd.read_csv('words/top1000_words.txt', sep = ',', header = -1).T[0]
words = np.array([x.replace(' ', '') for x in words])
words_vectors = tools.encode_sentences(model, words, verbose=True)
words_vectors_f30k = tools.encode_sentences(model_f30k, words, verbose=True)

1
1


#### Сохраняем представления для картинок в имаджинариуме и слов

In [10]:
import pickle
with open('preprocessed_data/words_coco_vectors.pickle', 'wb') as f:
    pickle.dump(words_vectors, f)
with open('preprocessed_data/image_coco_vectors.pickle', 'wb') as f:
    pickle.dump(image_vectors, f)